## Классификация объвлений по категориям

In [1]:
from lxml import etree
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import collections
%matplotlib inline

Загружаем данные

In [2]:
data_train = pd.read_csv("./train.csv")
data_train.head(5)

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [4]:
data_target = data_train["category_id"].values
corpus = data_train["title"].values
feature = data_train["price"].values

Разделяем даннын на тестовые и тренировочные(Feature отвечает за цены, X за текстовое представление, y за категории)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, Feature_train, Feature_test, y_train, y_test = train_test_split(corpus, feature, data_target, test_size=0.33, random_state=42)

Пробуем наивный CountVectorizer

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("algo", LogisticRegression(penalty='l1', C=17))
])

In [7]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [8]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.scorer import make_scorer
f_scorer = make_scorer(f1_score, average='macro')
print("Train_acc: {:.4f}, train_f-measure: {:.4f}".format(
    accuracy_score(pipeline.predict(X_train), y_train),
    f_scorer(pipeline, X_train, y_train)
))

Train_acc: 0.9459, train_f-measure: 0.9459


In [9]:
print("Test_acc: {:.4f}, test_f-measure: {:.4f}".format(
    accuracy_score(pipeline.predict(X_test), y_test),
    f_scorer(pipeline, X_test, y_test)
))

Test_acc: 0.8345, test_f-measure: 0.8369


Добавим знание о цене товара
Используюя RF классифицируем по цене

In [10]:
from sklearn.ensemble import RandomForestClassifier
algo_tree = RandomForestClassifier(n_estimators = 50, n_jobs=-1)
algo_tree.fit(Feature_train.reshape(-1,1), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
print(accuracy_score(algo_tree.predict(Feature_test.reshape(-1,1)), y_test))

0.08423248587046013


Не сильно большой Accuracy..
Попробуем добавить его к предыдущему результату 

In [34]:
price_pred = algo_tree.predict_proba(Feature_test.reshape(-1,1))
title_pred = pipeline.predict_proba(X_test)
pred_prob = price_pred + 3*title_pred
pred = [np.argmax(pred_prob[i]) for i in range(0, pred_prob.shape[0])]
print("Test_acc: {:.4f}".format(
    accuracy_score(pred, y_test),
))

Test_acc: 0.8350


### Посчитаем accuracy модели на каждом уровне иерархии.
Сначала по первому уровню

In [63]:
category_file = pd.read_csv("./category.csv")
dicte = [(category_file["name"][i].split("|")[0]) for i in range(0,len(category_file))]
print(np.unique(dicte))

['Бытовая электроника' 'Для дома и дачи' 'Личные вещи' 'Хобби и отдых']


In [58]:
pred_cat =  [dicte[pred[i]] for i in range(len(pred))]
y_test_cat = [dicte[y_test[i]] for i in range(len(y_test))]
print("Test_acc: {:.4f}".format(
    accuracy_score(pred_cat, y_test_cat),
))

Test_acc: 0.9317


Затем по второму уровню

In [66]:
dicte1 = [(category_file["name"][i].split("|")[1]) for i in range(0,len(category_file))]
print(np.unique(dicte1))

['Аудио и видео' 'Билеты и путешествия' 'Бытовая техника' 'Велосипеды'
 'Игры, приставки и программы' 'Книги и журналы' 'Коллекционирование'
 'Красота и здоровье' 'Мебель и интерьер' 'Музыкальные инструменты'
 'Настольные компьютеры' 'Ноутбуки' 'Одежда, обувь, аксессуары'
 'Охота и рыбалка' 'Планшеты и электронные книги'
 'Посуда и товары для кухни' 'Растения' 'Ремонт и строительство'
 'Спорт и отдых' 'Телефоны' 'Товары для детей и игрушки'
 'Товары для компьютера' 'Часы и украшения']


In [67]:
pred_cat =  [dicte1[pred[i]] for i in range(len(pred))]
y_test_cat = [dicte1[y_test[i]] for i in range(len(y_test))]
print("Test_acc: {:.4f}".format(
    accuracy_score(pred_cat, y_test_cat),
))

Test_acc: 0.8987


## Применяем к тестовым данным

In [68]:
data_train = pd.read_csv("./test.csv")
data_train.head(5)

,item_id,title,description,price
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
3,489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
4,489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


In [74]:
corpus = data_train["title"].values
feature = data_train["price"].values
item_id = data_train["item_id"].values

In [98]:
price_pred = algo_tree.predict_proba(feature.reshape(-1,1))
title_pred = pipeline.predict_proba(corpus)
pred_prob = price_pred + 3*title_pred
result = [np.argmax(pred_prob[i]) for i in range(0, pred_prob.shape[0])]
d3 = {'item_id': item_id, 'category_id': result}

In [99]:
df_result = pd.DataFrame.from_dict(d3)

In [100]:
df_result

,item_id,category_id
0,489517,22
1,489518,0
2,489519,15
3,489520,33
4,489521,13
5,489522,25
6,489523,50
7,489524,16
8,489525,6
9,489526,28


In [101]:
df_result.to_csv("result.csv", sep='\t')